# Finite Difference Stencils

We want to represent $f'(0)$ in the following form (say up to $O(h^5)$):
$$
f'(0) = \frac{c_1 f(-2h) + c_2 f(-h) + c_3 f(+h) + c_4 f(+2h)}{h}
$$
which should be ture for any analytic function, for the first few bases, we can choose:
$$
\begin{align}
f(x) = 1, f(x) = x, f(x) = x^2, f(x) = x^3.
\end{align}
$$
Take derivatives
$$
\begin{align}
\left.
\begin{array}{c}
\frac{c_1 + c_2 + c_3 + c_4}{h} &= 0,\\
\frac{c_1 (-2h) + c_2 (-h) + c_3 (h) + c_4 (h)}{h} &= 1,\\
\frac{c_1 (-2h)^2 + c_2 (-h)^2 + c_3 (h)^2 + c_4 (2h)^2}{h} &= 0,\\
\frac{c_1 (-2h)^3 + c_2 (-h)^3 + c_3 (h)^3 + c_4 (2h)^3}{h} &= 0,\\
\end{array}
\right\}
%\begin{array}{c}
%
%\end{array}
\end{align}
$$

In [1]:
include("stencils.jl")
include("../writeio.jl")
using .FDStencils
using .WriteIO
#
using Symbolics

In [4]:
@variables gf(..)

1-element Vector{Symbolics.CallWithMetadata{SymbolicUtils.FnType{Tuple, Real}, Base.ImmutableDict{DataType, Any}}}:
 gf⋆

In [12]:
simple = [-2, -1, 0, 1, 2];
values = [0, 1, 0, 0, 0];

In [14]:
coeff_C_1stD_2ndA = GetCoefficient(simple, values)

5-element Vector{Float64}:
  0.08333333333333348
 -0.6666666666666667
 -0.0
  0.6666666666666666
 -0.08333333333333333

In [39]:
C_1stD_2ndA = sum(coeff_C_1stD_2ndA[i] * gf(i-floor(Int, length(coeff_C_1stD_2ndA)/2+1)) for i in 1:length(coeff_C_1stD_2ndA))

0.6666666666666666gf(1) + 0.08333333333333348gf(-2) - 0.08333333333333333gf(2) - 0.6666666666666667gf(-1)

In [40]:
open("test.hxx", "w") do f
    print(f, string(C_1stD_2ndA))
end

## Write to Files

In [4]:
filename = "test.hxx";
thornname = "WaveToyHigherOrderX";

In [5]:
function headpart(pr::Function)
    pr("#ifndef " * replace(uppercase(filename), "."=>"_"))
    pr("#define " * replace(uppercase(filename), "."=>"_"))
    pr()
    pr("#include <cctk.h>")
    pr("namespace " * thornname * " {")
    pr()
end

headpart (generic function with 1 method)

In [6]:
function bodypart(pr::Function)
    pr(cs[1])
    pr(cs[2])
end

bodypart (generic function with 1 method)

In [7]:
function tailpart(pr::Function)
    pr("} // namespace " * thornname)
    pr("#endif // #ifndef " * replace(uppercase(filename), "."=>"_"))
end

tailpart (generic function with 1 method)

In [8]:
WriteFile(headpart, bodypart, tailpart, filename)

file test.hxx already, exist, replacing it ...
Done generating test.hxx
